In [ ]:
#importing all relevant libraries 
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib as plt 
#importing dataset
GFN_df = pd.read_csv('loan.csv')

In [ ]:
## DATA CLEANING
# Removing Non-Relevant Fields
GFN_df_ver1 = GFN_df.drop(['acc_open_past_24mths','avg_cur_bal','addr_state','all_util','annual_inc_joint','acc_now_delinq','bc_open_to_buy','bc_util','collections_12_mths_ex_med','delinq_amnt','desc','dti_joint','earliest_cr_line','emp_title','funded_amnt_inv','grade','home_ownership','il_util','initial_list_status','inq_fi','inq_last_12m','inq_last_6mths','installment','int_rate','issue_d','last_credit_pull_d','last_pymnt_amnt','pymnt_plan','last_pymnt_d','max_bal_bc','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_last_delinq','mths_since_last_major_derog','mths_since_last_record','mths_since_rcnt_il','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','next_pymnt_d','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','open_acc','open_acc_6m','open_il_12m','open_il_24m','open_il_6m','open_rv_12m','open_rv_24m','out_prncp','out_prncp_inv','pct_tl_nvr_dlq','percent_bc_gt_75','policy_code','pub_rec','pub_rec_bankruptcies','revol_util','revol_bal','recoveries','collection_recovery_fee','chargeoff_within_12_mths','sub_grade','tax_liens','title','tot_coll_amt','tot_hi_cred_lim','total_acc','total_bal_ex_mort','total_bal_il','total_bc_limit','total_cu_tl','tot_cur_bal','total_il_high_credit_limit','total_pymnt_inv','total_rec_int','total_rec_late_fee','total_rec_prncp','total_rev_hi_lim','url','verification_status','verification_status_joint','zip_code'], axis=1)

In [ ]:
## DATA CLEANING
# Replcaing Null Values with empty cells
GFN_df_ver1 = GFN_df_ver1.fillna('')

# Removing rows with loan status 'Current'
GFN_df_ver1 = GFN_df_ver1[GFN_df_ver1.loan_status != 'Current']

# Removing rows with applicant service length is less thatn 1 year
GFN_df_ver1 = GFN_df_ver1[GFN_df_ver1.emp_length != '< 1 year']

# Removing Rows where Employee service length is empty
GFN_df_ver1.drop(GFN_df_ver1.loc[GFN_df_ver1['emp_length']== ''].index, inplace=True)

# Replcaing 10+ years with 10 year as '+' is special character and won't allow for any operations on variable
GFN_df_ver1.loc[(GFN_df_ver1['emp_length'] == '10+ years'), 'emp_length'] = '10 years'

# Removing ' years' string from emp_length column so that we can get the data in month 
GFN_df_ver1['emp_length'] = GFN_df_ver1['emp_length'].map(lambda x: x.rstrip(' years'))

# Converting Employee service time from Year to Month
GFN_df_ver1['Emp_Service_Months'] = GFN_df_ver1.apply(lambda row: (int(row.emp_length) * 12), axis = 1) 

In [ ]:
## DATA ANALYSIS
# Box plot for annual income to identify outliers
print(GFN_df_ver1.boxplot(column='annual_inc'))

In [ ]:
from scipy.stats import zscore
# calculates z-score values
GFN_df_ver1["zscore"] = zscore(GFN_df_ver1["annual_inc"])

# Based on z-score outliers can be identified in annual income field
GFN_df_ver1["is_outlier"] = GFN_df_ver1["zscore"].apply(lambda x: x <= -1.96 or x >= 1.96)

In [ ]:
# Removing outliers rows as identified in previous code.
GFN_df_ver1 = GFN_df_ver1[GFN_df_ver1['annual_inc'].between(4000.0, 2039784.0)]

In [ ]:
# Getting Minimum, Maximum and mean values to identify buckets
aggre = np.array(GFN_df_ver1['annual_inc'])
print(aggre.mean(), aggre.min(), aggre.max())

In [ ]:
# Based on previous code, arbitary bucket defined and dataset is divided into 3 buckets
grades = []
for row in GFN_df_ver1['annual_inc']:
    if row <= 100000:    grades.append('Low_Income_Group')
    elif row > 100000 and row <= 1000000:  grades.append('Middle Income Group')
    else:           grades.append('High Income Group')
GFN_df_ver1['grades'] = grades

In [ ]:
# Arbitary bucket defined to identify risk based on employee delinq in last 2 years
Delinq_Risk = []
for row in GFN_df_ver1['delinq_2yrs']:
    if row == 0:    Delinq_Risk.append('Low Risk')
    elif row > 0 and row <= 5:  Delinq_Risk.append('Medium Risk')
    else:           Delinq_Risk.append('High Risk')
GFN_df_ver1['Delinq_Risk'] = Delinq_Risk

In [ ]:
## DATA VISUALIZATION
# Plot of previous defined grades against purpose of loan
import matplotlib.pyplot as plt
GFN_df_ver1.groupby(['grades','purpose']).size().unstack().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
# Plot of previous defined grades against Risk of Delinq
GFN_df_ver1.groupby(['grades','Delinq_Risk']).size().unstack().plot(kind='barh',stacked=True)
plt.show()

In [ ]:
# Plot of previous defined Risk of Delinq against term of loan
GFN_df_ver1.groupby(['term','delinq_2yrs']).size().unstack().plot(kind='bar',stacked=False)
plt.show()

In [ ]:
# Plot of previous defined Employee service month against term of loan
GFN_df_ver1.groupby(['Emp_Service_Months','term']).size().unstack().plot(kind='bar',stacked=False)
plt.show()

In [ ]:
# Taking Minimum, Maximum and Mean value of Debt to Income ratio
repay = np.array(GFN_df_ver1['dti'])
print(repay.mean(), repay.min(), repay.max())

In [ ]:
# Based on dti, employee is identified in 3 bucket of repayment ability
Repay = []
for row in GFN_df_ver1['dti']:
    if row <= 15:    Repay.append('High')
    elif row > 15 and row <= 25:  Repay.append('Medium')
    else:           Repay.append('Low')
GFN_df_ver1['Likely_To_Repay'] = Repay

In [ ]:
# Finally only relevant fields available to make the decision
GFN_df_ver1 = GFN_df_ver1.drop(['member_id','loan_amnt','funded_amnt','term','emp_length','annual_inc','loan_status','purpose','dti','delinq_2yrs','total_pymnt','application_type','Emp_Service_Months','zscore','is_outlier'], axis=1)